<a href="https://colab.research.google.com/github/alanParizek/MachineLearningFCC/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from tensorflow.keras.layers import TextVectorization
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
import csv

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_ds = pd.read_csv('train-data.tsv', sep='\t', header=None)
test_ds = pd.read_csv('valid-data.tsv', sep='\t', header=None)

train_ds.columns = ['type', 'message']
test_ds.columns = ['type', 'message']

train_ds['type'] = pd.Categorical(train_ds['type'])
test_ds['type'] = pd.Categorical(test_ds['type'])

#i pad or truncate the messages so they are 250 chars each
train_ds['message'] = train_ds['message'].str.zfill(250)
train_ds['message'] = train_ds['message'].str.slice(0,250)


train_labels = train_ds.type.cat.codes
train_ds.pop('type')

test_labels = test_ds.type.cat.codes
test_ds.pop('type')

test_ds['message'] = test_ds['message'].str.zfill(250)
test_ds['message'] = test_ds['message'].str.slice(0,250)

train_ds.head()
train_labels.head()

In [ ]:
#counting how many individual words are in the dataset

# Combine all text in the 'message' column into one long string
all_text = ' '.join(train_ds['message'].dropna().astype(str))

# Split into words and count unique ones
word_list = all_text.split()
unique_words = set(word_list)
num_unique_words = len(unique_words)

print("Number of unique words:", num_unique_words)

In [ ]:
#encoding messages into floats

vectorizer = TextVectorization(max_tokens=num_unique_words, output_sequence_length=250)

# Adapt to your text data
vectorizer.adapt(train_ds)
vectorizer.adapt(test_ds)

# Vectorize the training text
train_ds_vectorized = vectorizer(train_ds)
test_ds_vectorized = vectorizer(train_ds)

In [ ]:
#building
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_unique_words, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
print(type(train_ds))
print(type(train_labels))

In [ ]:
model.summary()

In [ ]:
#training
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_ds_vectorized, train_labels, epochs=10, validation_split=0.2)

In [ ]:
#testing

results = model.evaluate(test_ds_vectorized, test_labels)
print(results)

In [ ]:
def encode_text(text):
  return vectorizer(text)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return ()

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
